In [1]:
#!pip install git+https://github.com/EasyArray/ling516.git@v4

In [2]:
%config InteractiveShell.ast_node_interactivity='all'
from phosphorus import *


             _    _                  _    _
            | |  | |                | |  | |
           _| |_ | |__   ___  ___  _| |_ | |__   ___  _ __ _   _  ____
          /     \| '_ \ / _ \/ __|/     \| '_ \ / _ \| '__| | | |/ ___)
         ( (| |) ) | | | (_) \__ ( (| |) ) | | | (_) | |  | |_| ( (__
          \_   _/|_| |_|\___/|___/\_   _/|_| |_|\___/|_|   \__,_|\__ \
            | |                     | |                            _) )
            |_|                     |_|                           (__/

        Welcome to the Phosphorus Meaning Engine v4
        Created by Ezra Keshet (EzraKeshet.com)




In [3]:
`LAUGHS(A)
`LAUGHS(A).t
`LAUGHS(A) and CRIES(A)
`lambda x: LAUGHS(x)
`lambda x=e: LAUGHS(x)
`lambda x=e: LAUGHS(x).t
`lambda x=e: LAUGHS(x) and CRIES(x)

LAUGHS(A)

LAUGHS(A)

LAUGHS(A) and CRIES(A)

lambda x: LAUGHS(x)

lambda x: LAUGHS(x)

lambda x: LAUGHS(x)

lambda x: LAUGHS(x) and CRIES(x)

In [4]:
for x in DOMAIN[:4]: display(x)

'A'

'B'

'C'

'D'

In [5]:
A,B,C,D,*_ = DOMAIN
`(lambda x=e: LAUGHS(x).t)(A)
A

LAUGHS('A')

'A'